<a href="https://colab.research.google.com/github/lce2005/lotto_number_prediction_program/blob/main/%EB%A1%9C%EB%98%90%EB%B2%88%ED%98%B8%EC%98%88%EC%B8%A1%ED%94%84%EB%A1%9C%EA%B7%B8%EB%9E%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
#로또번호 예측 트랜스포머

import torch
import torch.nn as nn
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader
import numpy as np

# CSV 파일 불러오기
data = pd.read_csv('lotto.csv')

# 데이터 전처리
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data.iloc[:, 1:].values)  # 날짜 열 제외, 수치 데이터 스케일링

# 훈련 데이터 준비
train_data = scaled_data[:-6]  # 마지막 6개 행 제외
train_inputs = torch.tensor(train_data[:-1]).unsqueeze(0).float()  # 입력 데이터
train_labels = torch.tensor(train_data[1:]).unsqueeze(0).float()  # 출력 데이터


# 데이터셋 클래스 정의
class LottoDataset(torch.utils.data.Dataset):
    def __init__(self, inputs, labels):
        self.inputs = inputs
        self.labels = labels

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, index):
        return self.inputs[index], self.labels[index]

# 트랜스포머 모델 정의
class LottoTransformer(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers, num_heads):
        super(LottoTransformer, self).__init__()
        self.encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(input_size, num_heads, hidden_size),
            num_layers
        )
        self.decoder = nn.Linear(input_size, output_size)

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

# 하이퍼파라미터 설정
input_size = train_inputs.shape[-1]
hidden_size = 64
output_size = train_labels.shape[-1]
num_layers = 4
num_heads = 4
num_epochs = 1000
learning_rate = 0.001
batch_size = 32

# 데이터 로더 초기화
dataset = LottoDataset(train_inputs, train_labels)
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# 모델 초기화
model = LottoTransformer(input_size, hidden_size, output_size, num_layers, num_heads)

# 손실 함수와 옵티마이저 정의
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# 모델 훈련
for epoch in range(num_epochs):
    model.train()
    for batch_inputs, batch_labels in data_loader:
        optimizer.zero_grad()
        outputs = model(batch_inputs)
        loss = criterion(outputs, batch_labels)
        loss.backward()
        optimizer.step()
    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# 테스트 데이터 준비
test_inputs = torch.tensor(scaled_data[-3:-1]).unsqueeze(0).float()

# 모델 예측
model.eval()
with torch.no_grad():
    predicted = model(test_inputs)

# 스케일링 역변환
predicted = scaler.inverse_transform(predicted.squeeze(0).numpy())

# 예측 결과 출력
lotto_numbers_prediction = predicted[-1].astype(int)[:-5]  # 마지막 5개 원소 제외
print(f'2025년 12월 로또 번호 예측: {lotto_numbers_prediction}')

/usr/local/lib/python3.12/dist-packages/torch/nn/modules/transformer.py:392: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


Epoch [100/1000], Loss: 0.0361
Epoch [200/1000], Loss: 0.0303
Epoch [300/1000], Loss: 0.0285
Epoch [400/1000], Loss: 0.0279
Epoch [500/1000], Loss: 0.0275
Epoch [600/1000], Loss: 0.0274
Epoch [700/1000], Loss: 0.0272
Epoch [800/1000], Loss: 0.0271
Epoch [900/1000], Loss: 0.0270
Epoch [1000/1000], Loss: 0.0269
2025년 12월 로또 번호 예측: [ 6 12 20 26 32 39 22]
